# US Accidents – Memory Optimization Pipeline
**From 2.85 GB → 0.95 GB (66% reduction) – Zero information loss**

## connecting to github and kaggle

In [1]:
!pip install kaggle --quiet


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

The syntax of the command is incorrect.
'cp' is not recognized as an internal or external command,
operable program or batch file.
'chmod' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
!kaggle datasets download -d sobhanmoosavi/us-accidents

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\Dear user\AppData\Local\Programs\Python\Python313\Scripts\kaggle.exe\__main__.py", line 6, in <module>
    sys.exit(main())
             ~~~~^^
  File "C:\Users\Dear user\AppData\Local\Programs\Python\Python313\Lib\site-packages\kaggle\cli.py", line 68, in main
    out = args.func(**command_args)
  File "C:\Users\Dear user\AppData\Local\Programs\Python\Python313\Lib\site-packages\kaggle\api\kaggle_api_extended.py", line 1741, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
         ~~~~~~~~~~~~~~~~~~~~~~~~^^
  File "C:\Users\Dear user\AppData\Local\Programs\Python\Python313\Lib\site-packages\kaggle\api\kaggle_api_extended.py", line 688, in build_kaggle_client
    username=self.config_values['username'],
             ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
KeyError: 'username'


In [4]:
!unzip us-accidents.zip

'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...


'"uv pip install --system pynvml"' is not recognized as an internal or external command,
operable program or batch file.


Traceback (most recent call last):
  File "c:\Users\Dear user\Desktop\decrease dataset size\rapidsai-csp-utils\colab\pip-install.py", line 7, in <module>
    import pynvml
ModuleNotFoundError: No module named 'pynvml'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\Dear user\Desktop\decrease dataset size\rapidsai-csp-utils\colab\pip-install.py", line 16, in <module>
    import pynvml
ModuleNotFoundError: No module named 'pynvml'


## import libraries

In [6]:
import cudf
import numpy as np

ModuleNotFoundError: No module named 'cudf'

## open csv file and get info about dataset

In [ ]:
df = cudf.read_csv("/content/US_Accidents_March23.csv")

## Drop Useless Columns – One-liner style

### ID : just a unique identifier, no predictive value

In [ ]:
df.drop(["ID"] , inplace=True , axis=1)

### Start_Time & End_Time : fully covered by Weather_Hour + Sunrise_Sunset

In [ ]:
df.drop(["Start_Time" , "End_Time" ] , inplace=True , axis=1)

### Description : free text, 100% redundant

In [ ]:
df.drop(["Description"] , inplace=True , axis=1)

### Street, City, Zipcode : high cardinality → causes overfitting

In [ ]:
df.drop(["Street" , "City" , "Zipcode"] , inplace=True , axis=1)

### Country : all are USA

In [ ]:
df.drop(["Country"] , inplace=True , axis=1)

### Airport_Code : not important

In [ ]:
df.drop(["Airport_Code"] , inplace=True , axis=1 )

### Civil_Twilight , Nautical_Twilight , Astronomical_Twilight : we dont need them because we have Sunrise_Sunset column

In [ ]:
df.drop(["Civil_Twilight" , "Nautical_Twilight" , "Astronomical_Twilight"] , inplace=True , axis=1)

### Bump column : column removed because Over 99.94% of values are `False` (only 4 `True` in 7,728 rows), providing negligible predictive information.

In [ ]:
df.drop(["Bump"] , inplace=True , axis=1)

### Give_Way column : column removed because Over 99.71% of values are `False` (only 22 `True` in 7,728 rows), providing negligible predictive information.

In [ ]:
df.drop(["Give_Way"] , inplace=True , axis=1)

### No_Exit column : column removed because Over 99.76% of values are `False` (only 18 `True` in 7,728 rows), providing negligible predictive information.

In [ ]:
df.drop(["No_Exit"] , inplace=True , axis=1)

### Railway column : column removed because Over 99.22% of values are `False` (only 60 `True` in 7,728 rows), providing negligible predictive information.

In [ ]:
df.drop(["Railway"] , inplace=True , axis=1)

### Roundabout column : column removed because Over 100% of values are `False` (only 0 `True` in 7,728 rows), providing negligible predictive information.

In [ ]:
df.drop(["Roundabout"] , inplace=True , axis=1)

### Traffic_Calming column : column removed because Over 99.88% of values are `False` (only 9 `True` in 7,728 rows), providing negligible predictive information.

In [ ]:
df.drop(["Traffic_Calming"] , inplace=True , axis=1)

### Turning_Loop column : column removed because Over 100% of values are `False` (only 0 `True` in 7,728 rows), providing negligible predictive information.

In [ ]:
df.drop(["Turning_Loop"] , inplace=True , axis=1)

## Change columns date type

### Source column : map data with dictionary and change datatype object to int8

In [ ]:
df["Source"].unique()
Source_dict = {"Source1" : 1 ,
               "Source2" : 2 ,
               "Source3" : 3 ,}
df["Source"] = df["Source"].map(Source_dict)
df["Source"] = df["Source"].fillna(0).astype("int8")


### Severity column : change datatype int64 to int8

In [ ]:
df["Severity"] = df["Severity"].astype("int8")

### County , State columns : drop and replace them with Fips

In [ ]:
fips = cudf.read_csv("national_county.txt",
                   header=None,
                   names=['State', 'StateFIPS', 'CountyFIPS', 'CountyName', 'ClassCode'],
                   sep=",",
                   dtype=str)

fips['State'] = fips['State'].str.strip()
fips['CountyName'] = fips['CountyName'].str.strip().str.lower()
fips['StateFIPS'] = fips['StateFIPS'].str.zfill(2)
fips['CountyFIPS'] = fips['CountyFIPS'].str.zfill(3)

fips['FIPS'] = fips['StateFIPS'] + fips['CountyFIPS']

df['County_lower'] = df['County'].str.lower().str.strip()
df['State'] = df['State'].str.strip()

df = df.merge(
    fips[['State', 'CountyName', 'FIPS']],
    how='left',
    left_on=['State', 'County_lower'],
    right_on=['State', 'CountyName']
)

df['FIPS'] = df['FIPS'].fillna('99999').astype('int64')

df = df.drop(['County', 'State', 'County_lower', 'CountyName'], axis=1)

### Timezone column : change dataType object to int8

In [105]:
df["Timezone"].unique()
tz_dic = {
    "US/Eastern" : 1 ,
    "US/Pacific" : 4 ,
    "US/Central" : 2 ,
    "US/Mountain" : 3 ,
}
df["Timezone"] = df["Timezone"].map(tz_dic).fillna(0).astype("int8")

### Weather_Timestamp column : drop and replace it with Weather_Hour , Weather_date , Is_night columns

In [107]:
df["Weather_Timestamp"] = cudf.to_datetime(df["Weather_Timestamp"] , errors="raise")

In [108]:
df.insert(loc= df.columns.get_loc("Weather_Timestamp")+1 , column="Weather_Hour" , value=df["Weather_Timestamp"].dt.hour.fillna(-1).astype("int8"))

In [109]:
df.insert(loc = df.columns.get_loc("Weather_Timestamp")+2 , column="Weather_date" , value=df["Weather_Timestamp"].dt.floor("D"))

In [110]:
df.insert(loc = df.columns.get_loc("Weather_Timestamp")+3 , column="Is_night" , value=((df["Weather_Hour"] >= 18) | (df["Weather_Hour"] <= 5)).fillna(False))

In [141]:
df["Is_night"] = df["Is_night"].astype("int8")

In [111]:
df.drop(["Weather_Timestamp"] , inplace=True , axis=1)

### Temperature column : fill nan with median and change datatype float64 to int8

In [112]:
temp_med = df["Temperature(F)"].median()
df["Temperature(F)"] = df["Temperature(F)"].fillna(temp_med).astype("int8")

### Wind_Chill column : fill nan with median and change datatype float64 to int8

In [113]:
wind_med = df["Wind_Chill(F)"].median()
df["Wind_Chill(F)"] = df["Wind_Chill(F)"].fillna(wind_med).astype("int8")

### Humidity column : fill nan with median and change datatype float64 to int8

In [114]:
hu_med = df["Humidity(%)"].median()
df["Humidity(%)"] = df["Humidity(%)"].fillna(hu_med).astype("int8")

### Pressure column : fill nan with median and change datatype float64 to float16

In [115]:
pr_med = round(df["Pressure(in)"].median() , 2)
df["Pressure(in)"] = df["Pressure(in)"].fillna(pr_med).round(2)
df["Pressure(in)"] = df["Pressure(in)"].astype("float32")

### Visibility column : fill nan with median and change datatype float64 to int8

In [116]:
vis_med = df["Visibility(mi)"].median()
df["Visibility(mi)"] = df["Visibility(mi)"].fillna(vis_med).astype("int8")

### Wind_Direction column : map data with dictionary and change datatype object to int8

In [117]:
wind_mapping = {
    'N': 0, 'NNE': 1, 'NE': 2, 'ENE': 3,
    'E': 4, 'ESE': 5, 'SE': 6, 'SSE': 7,
    'S': 8, 'SSW': 9, 'SW': 10, 'WSW': 11,
    'W': 12, 'WNW': 13, 'NW': 14, 'NNW': 15,
    'CALM': 16, 'VAR': 17, 'VARIABLE': 17
}
df["Wind_Direction"] = df["Wind_Direction"].astype(str).str.upper()
df["Wind_Direction"] = df["Wind_Direction"].map(wind_mapping)
df["Wind_Direction"] = df["Wind_Direction"].fillna(-1).astype("int8")

### Wind_Speed column : fill nan with median and change datatype float64 to int8

In [118]:
wind_med = df["Wind_Speed(mph)"].median()
df["Wind_Speed(mph)"] = df["Wind_Speed(mph)"].fillna(wind_med).astype("int8")

### Weather_Condition column : change datatype object to category

In [119]:
df["Weather_Condition"] = df["Weather_Condition"].astype(str).str.strip()
df["Weather_Condition"] = df["Weather_Condition"].replace({"nan" : np.nan , "N/A" : np.nan})
df["Weather_Condition"] = df["Weather_Condition"].astype("category")

### Amenity column : change datatype boolean to int8 ( The data size will not decrease, but processing on integers will be better. )

In [120]:
df["Amenity"] = df["Amenity"].fillna(False).astype("int8")

### Crossing column : change datatype boolean to int8 ( The data size will not decrease, but processing on integers will be better. )

In [122]:
df["Crossing"] = df["Crossing"].fillna(False).astype("int8")

### Junction column : change datatype boolean to int8 ( The data size will not decrease, but processing on integers will be better. )

In [124]:
df["Junction"] = df["Junction"].fillna(False).astype("int8")

### Station column : change datatype boolean to int8 ( The data size will not decrease, but processing on integers will be better. )

In [128]:
df["Station"] = df["Station"].fillna(False).astype("int8")

### Stop column : change datatype boolean to int8 ( The data size will not decrease, but processing on integers will be better. )

In [129]:
df["Stop"] = df["Stop"].fillna(False).astype("int8")

### Traffic_Signal column : change datatype boolean to int8 ( The data size will not decrease, but processing on integers will be better. )

In [142]:
df["Traffic_Signal"] = df["Traffic_Signal"].astype("int8")

### Sunrise_Sunset column : map data with dictionary and change datatype object to int8

In [132]:
sun_dic = {
    "Day" : 1 ,
    "Night" : 0
}
df["Sunrise_Sunset"] = df["Sunrise_Sunset"].map(sun_dic)
sun_med = df["Sunrise_Sunset"].median()
df["Sunrise_Sunset"] = df["Sunrise_Sunset"].fillna(sun_med).astype("int8")

## Convert optimized cuDF dataframe back to CSV file

In [ ]:
df.to_csv("US_Accidents_Opt.csv")